In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/data_transformed.csv")

In [3]:
df.isna().sum()

event-id                        0
location-long                   0
location-lat                    0
argos:lat1                      0
argos:lat2                     85
argos:lon1                      0
argos:lon2                     85
individual-local-identifier     0
date                            0
time                            0
dtype: int64

In [4]:
df = df.dropna()

In [5]:
df.isna().sum()

event-id                       0
location-long                  0
location-lat                   0
argos:lat1                     0
argos:lat2                     0
argos:lon1                     0
argos:lon2                     0
individual-local-identifier    0
date                           0
time                           0
dtype: int64

In [6]:
import datetime as dt

In [7]:
df["year"] = df["date"].apply(lambda x: 
                                    dt.datetime.strptime(x, "%Y-%m-%d").year)


In [8]:
df["month"] = df["date"].apply(lambda x: 
                                    dt.datetime.strptime(x, "%Y-%m-%d").month)

In [9]:
df=df.drop(columns=["event-id", "date", "individual-local-identifier"])

In [10]:
df

,location-long,location-lat,argos:lat1,argos:lat2,argos:lon1,argos:lon2,time,year,month
0,-122.407,36.981,36.981,47.285,-122.407,-69.471,01:23:17,1993,8
1,-122.788,37.637,37.637,39.596,-122.788,-132.219,13:04:34,1993,8
2,-122.827,37.166,37.166,44.267,-122.827,-86.243,01:57:39,1993,9
3,-123.466,38.184,38.184,45.262,-123.466,-88.640,22:35:12,1993,9
4,-123.410,37.141,37.141,34.968,-123.410,-133.827,03:40:17,1993,9
...,...,...,...,...,...,...,...,...,...
15540,-115.027,26.702,26.702,37.012,-115.027,-165.932,19:13:04,2008,6
15541,-115.930,26.972,26.972,18.266,-115.930,-154.364,22:27:15,2008,6
15542,-119.121,30.734,30.734,21.285,-119.121,-73.036,11:42:10,2008,6
15543,-119.172,30.742,30.742,23.543,-119.172,-86.128,11:59:00,2008,6
